# Challenge csv user friendly

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from satstac import Catalog
from satstac import Collection
from satstac import Item
import rasterio
import shapely
import sys,os,os.path
import json

In [2]:
df = pd.read_csv('data_to_STAC.csv')
df.head()


,collection,part,id,datetime,td:title,td:description,td:type,td:classes,asset:raster:title,asset:raster:href,...,asset:thumbnail:type,asset:chips:title,asset:chips:href,asset:chips:type,asset:notebook:title,asset:notebook:href,asset:notebook:type,bbox,geometry,additional_keywords
0,accra,accra_1,accra_1_buildings,2019-04-23T00:00:00Z,accra 1 building labels,NaN,segmentation,"['buildings', 'drains', 'roads' ]",accra_1_cog,https://oin-hotosm.s3.amazonaws.com/5bb9323e9e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,accra,accra_1,accra_1_drains,2019-04-23T00:00:00Z,accra 1 drain labels,NaN,segmentation,NaN,accra_1_cog,https://oin-hotosm.s3.amazonaws.com/5bb9323e9e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,accra,accra_1,accra_1_roads,2019-04-23T00:00:00Z,accra 1road labels,some random description here,segmentation,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,monrovia,monrovia_2,monrovia_2_buildings-item,2019-04-23T00:00:00Z,monrovia 2 building labels,some random description here,segmentation,NaN,monrovia_2_image,https://oin-hotosm.s3.amazonaws.com/5b83a514c8...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,zanzibar,znz_022,znz022_buildings-item,2019-04-23T00:00:00Z,znz022 buildings,some random description here,"segmentation, classification",NaN,znz022_image,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
stac_version = '0.6.2'
stac_challenge_folder = './'#'./STAC_challenge_5'
catalog_address =  'catalog.json'

tiffiles = {"accra_1": "https://oin-hotosm.s3.amazonaws.com/5b694a0f4b87366cc0f0fa70/0/5b694a0f4b8736ebfff0fa71.tif",
           "accra_2":"https://oin-hotosm.s3.amazonaws.com/5bb9323e9ed15b0006d24f33/0/5bb9323e9ed15b0006d24f34.tif",
           'accra_3':'https://oin-hotosm.s3.amazonaws.com/5be9bb18080ac000051474fd/0/5be9bb18080ac000051474fe.tif',
           'monrovia_1':'https://oin-hotosm.s3.amazonaws.com/5c08c2ec6918390006b7a8a1/0/5c08c2ec6918390006b7a8a2.tif',
           'monrovia_2':'https://oin-hotosm.s3.amazonaws.com/5b83a514c8e197000a93403e/0/5b83a514c8e197000a93403f.tif',
           'monrovia_3':'https://oin-hotosm.s3.amazonaws.com/5bcdce33b9e5f20005f7da3e/0/5bcdce33b9e5f20005f7da3f.tif',
           'monrovia_4':'https://oin-hotosm.s3.amazonaws.com/5b8180e87343a943f0347d18/0/5b8180e87343a991c8347d19.tif',
           'pointe-noire_1': 'https://oin-hotosm.s3.amazonaws.com/5c30a233be6ca30005c74da8/1/5c30a233be6ca30005c74daa.tif',
           'pointe-noire_2': 'https://oin-hotosm.s3.amazonaws.com/5c30a233be6ca30005c74da8/0/5c30a233be6ca30005c74da9.tif'
           }

### Functions for the items


In [68]:
#item's generic metadata: 
type_item = "Feature"
properties_item = {
    "datetime": "2019-02-26T00:00:00Z",
    "td:title": "test",
    "td:description": "test",
    "td:label_type": "segmentation",
    "td:classes": [
      'buildings', 'drains', 'roads'
    ]
  }
basic_keywords_items = ["challenge", "world bank"]

In [69]:
def get_bbox(address):
    gdf = gpd.read_file(address);
    gdf = gdf[gdf['geometry'].isna() != True];
    return list(gdf['geometry'].total_bounds)

def get_bbox_csv(row):
    if(not(row.isnull()['bbox'])):
        return row['bbox'];
    elif(not(row.isnull()['asset:vector:href'])):
        gdf = gpd.read_file(row['asset:vector:href']);
        gdf = gdf[gdf['geometry'].isna() != True];
        return list(gdf['geometry'].total_bounds)
    elif(row.isnull()['asset:vector:href']):
        for dirpath, subdirs, files in os.walk(stac_challenge_folder):
            for f in files:
                if(f == row['id']+'.geojson'):
                    get_bbox(os.path.join(dirpath, f))   
    else:
        return [-90,90,-180,180]

def get_bbox_local(address):
    return get_bbox(address)
        

In [70]:
get_bbox_local(df.iloc[1]['asset:vector:href'])

[-0.2426018, 5.6204321, -0.2287005, 5.645931]

In [71]:
get_bbox_csv(df.iloc[5])

[39.28919876472999, -5.743028283012867, 39.31302874892266, -5.722212794937691]

In [72]:
def adapt_item_properties_csv(row):
    properties = properties_item.copy();
    if(not(row.isnull()['datetime'])):
        properties['datetime']= row['datetime'];
    if(not(row.isnull()['td:title'])):
        properties['td:title']= row['td:title'];
    else:
        properties['td:title']= row['id'];
    if(not(row.isnull()['td:description'])):
        properties['td:description']= row['td:description'];
    else:
        properties['td:description']=row['id']+' from '+row['collection']
    if(not(row.isnull()['td:type'])):
        properties['td:type']= row['td:type'];
    if(not(row.isnull()['td:classes'])):
        properties['td:classes']= row['td:classes'];
    if(not(row.isnull()['additional_keywords'])):
        properties['keywords']= row['additional_keywords'];
    return properties

       
def adapt_item_properties_local(filename, foldername):
    properties = properties_item.copy();
    properties['td:title']= filename;
    properties['td:description']= filename+' from '+foldername; 
    properties['keywords']= adapt_item_keywords_local(filename, foldername);
    return properties 

In [73]:
df.columns.unique()

Index(['collection', 'part', 'id', 'datetime', 'td:title', 'td:description',
       'td:type', 'td:classes', 'asset:raster:title', 'asset:raster:href',
       'asset:raster:type', 'asset:vector:title', 'asset:vector:href',
       'asset:vector:type', 'asset:thumbnail:title', 'asset:thumbnail:href',
       'asset:thumbnail:type', 'asset:chips:title', 'asset:chips:href',
       'asset:chips:type', 'asset:notebook:title', 'asset:notebook:href',
       'asset:notebook:type', 'bbox', 'geometry', 'additional_keywords'],
      dtype='object')

In [74]:
adapt_item_properties_csv(df.iloc[5])

{'datetime': '2019-04-23T00:00:00Z',
 'td:title': 'znz001 buildings',
 'td:description': 'some random description here',
 'td:label_type': 'segmentation',
 'td:classes': ['buildings', 'drains', 'roads'],
 'td:type': 'segmentation, classification'}

In [75]:
adapt_item_properties_local('accra_1_buildings', 'accra', [])

TypeError: adapt_item_properties_local() takes 2 positional arguments but 3 were given

In [ ]:
def adapt_item_keywords_csv(row):
    keywords = basic_keywords_items.copy();
    if(not(row.isnull()['additional_keywords'])):
        for k in row['additional_keywords']:
            keywords.append(k);
    else:
        keywords.append(row['id']);
        keywords.append(row['collection']);
    return keywords
    
def adapt_item_keywords_local(filename, foldername):
    keywords = basic_keywords_items.copy();
    keywords.append(filename);
    keywords.append(foldername);
    return keywords

In [ ]:
def get_geometry_csv(row):
    if(not(row.isnull()['geometry'])):
        geometry_item = row['geometry']
    if(not(row.isnull()['asset:vector:href'])):
        geometry_item = get_geometry(row['asset:vector:href']);
    else:
        for dirpath, subdirs, files in os.walk(stac_challenge_folder):
            for f in files:
                if(f == row['id']+'.geojson'):
                    geometry_item = get_geometry(os.path.join(dirpath, f))
    return geometry_item


        
        
def get_geometry(address):
    data = gpd.read_file(address); 
    geom = []
    for f in data['geometry']:
        if(f is not None):
            #We now have to know wether it is a multipolygon or polygon or multiLineString or LineString,
            #in order to get the complete geometry and not only the first polygon/line
            if('polygon' in str(type(data['geometry'][0])) or 'polygon' in str(type(data['geometry'][1]))):   
                geom_type = 'Polygon'
                if('MultiPolygon' in str(type(data['geometry'][0]))):   
                    polygons = list(f)
                    for po in polygons:
                        x, y = po.exterior.coords.xy
                        geom.append([[[x[i],y[i]] for i in range(len(x))]])
                else:
                    x, y = f.exterior.coords.xy
                    geom.append([[[x[i],y[i]] for i in range(len(x))]])
            if('line' in str(type(data['geometry'][0])) or 'line' in str(type(data['geometry'][1]))):
                geom_type = 'LineString'
                if('Multi' in str(type(data['geometry'][0]))):   
                    lines = list(f)
                    for li in lines:
                        x, y = li.coords.xy
                        geom.append([[x[i],y[i]] for i in range(len(x))])
                else:
                    x, y = f.coords.xy
                    geom.append([[x[i],y[i]] for i in range(len(x))]) 
            if('point' in str(type(data['geometry'][0])) or 'point' in str(type(data['geometry'][1]))):
                geom_type = 'LineString'
                if('Multi' in str(type(data['geometry'][0]))):   
                    points = list(f)
                    for po in points:
                        x, y = li.coords.xy
                        geom.append([x[i],y[i]] for i in range(len(x)))
                else:
                    x, y = f.coords.xy
                    geom.append([x[i],y[i]] for i in range(len(x))) 
    geometry_item = {
        "type": "Multi" + geom_type,
        "coordinates": geom
        } 
    return geometry_item

In [ ]:
get_geometry_csv(df.iloc[1])

In [ ]:
get_geometry('/Users/axel/Desktop/STAC_challenge_5/accra/accra_1/accra_1_drains.geojson')

In [ ]:
#this one can be done more efficiently if we decide to enforce having only title, href and type for each asset, in that case we don't have tpo go through row.columns the second time
def get_assets_csv(row):
    added = []
    assets_item = {}
    for l in row.axes[0]:
        if('asset'in l):
            if(not row.isnull()[l]):
                name = l.split(':')[1];
                if(name not in added):
                    added.append(name)
                    assets_item[name]={}
                    for l_2 in row.axes[0]:
                        if('asset:'+name in l_2):
                            assets_item[name][l_2.split(':')[2]] = row[l_2];
                    if(name == 'vector'):
                        keys = assets_item[name].keys()
                        if('title' not in keys):
                            assets_item[name]['title'] = 'vector'+row['id']+' '+row['collection'] ;
                        if('href' not in keys):
                            assets_item[name]['title'] = 'no_href' ;
                        if('type' not in keys):
                            assets_item[name]['title'] = 'application/geo+json' ;
                    if(name == 'image'):
                        keys = assets_item[name].keys()
                        if('title' not in keys):
                            assets_item[name]['title'] = 'vector'+row['id']+' '+row['collection'] ;
                        if('href' not in keys):
                            assets_item[name]['title'] = 'no_href' ;
                        if('type' not in keys):
                            assets_item[name]['title'] = 'image/vnd.stac.geotiff; cloud-optimized=true' ;
    if('vector' not in added):
        print('scroll through directory to find the vector')
        for dirpath, subdirs, files in os.walk(stac_challenge_folder):
            for f in files:
                if(f == row['id']+'.geojson'):
                    assets_item['vector']= {
                        "title": row['id'],
                        "href": os.path.join(dirpath, f),
                        "type": "application/geo+json"
                    }    
    return assets_item;  






def get_assets_local(filename, foldername, surfoldername):
    return {
    "raster": {
        "title": "image",
        "href": tiffiles[filename.split('_')[0]+'_'+filename.split('_')[1]],
        "type": "image/vnd.stac.geotiff; cloud-optimized=true"
    },
    "vector": {
      "title": filename.split('.')[0],
      "href": '../../'+surfoldername +'/'+foldername+'/'+filename,
      "type": "application/geo+json"
    }
    }
    
    

In [ ]:
get_assets_csv(df.iloc[0])

In [76]:
get_assets_local('accra_1_buildings.geojson','accra_1','accra')

{'raster': {'title': 'image',
  'href': 'https://oin-hotosm.s3.amazonaws.com/5b694a0f4b87366cc0f0fa70/0/5b694a0f4b8736ebfff0fa71.tif',
  'type': 'image/vnd.stac.geotiff; cloud-optimized=true'},
 'vector': {'title': 'accra_1_buildings',
  'href': '../../accra/accra_1/accra_1_buildings.geojson',
  'type': 'application/geo+json'}}

In [77]:
def get_tags_items_csv(row):
    links_item = [ {
      "rel": "self",
      "href": './'+'challenge_collection_'+ row['collection']+'/'+ row['part']+'/'+row['id']+'.json'
    },
    {
      "rel": "root",
      "href": '../../catalog.json'
    },]    
    #assets_item = get_assets_csv(row);
    #keywords_item = adapt_item_keywords_csv(row);
    #bbox_item = get_bbox_csv(row);
    #geometry_item = get_geometry_csv(row);
    #adapt_item_keywords_csv(row)
    
    return row['id'], type_item, get_geometry_csv(row),get_bbox_csv(row), adapt_item_properties_csv(row), links_item, get_assets_csv(row)
    
def get_tags_items_local(filename, foldername, surfoldername):
    links_item = [ {
      "rel": "self",
      "href": './'+'challenge_collection_'+ surfoldername+'/'+ foldername+'/'+filename.split('.')[0]+'-item.json'
    },
    {
      "rel": "root",
      "href": '../../catalog.json'
    },]  
    address = './'+surfoldername+'/'+foldername+'/'+filename
    #adapt_item_keywords_local(filename,foldername)
    return filename.split('.')[0]+'-item', type_item,get_geometry(address), get_bbox_local(address),adapt_item_properties_local(filename, foldername),links_item, get_assets_local(filename, foldername, surfoldername)
    
    
    

    
    

In [78]:
get_tags_items_local('accra_1_buildings.geojson','accra_1','accra')

('accra_1_buildings-item',
 'Feature',
 {'type': 'MultiPolygon',
  'coordinates': [[[[-0.2418751, 5.6395062],
     [-0.2418082, 5.639549],
     [-0.2417837, 5.6395112],
     [-0.2416979, 5.6395661],
     [-0.2417263, 5.6396101],
     [-0.2416545, 5.639656],
     [-0.2414883, 5.639399],
     [-0.2415592, 5.6393536],
     [-0.2415874, 5.6393972],
     [-0.2416806, 5.6393375],
     [-0.2416535, 5.6392955],
     [-0.2417139, 5.6392569],
     [-0.2418751, 5.6395062]]],
   [[[-0.2368022, 5.6383851],
     [-0.2366147, 5.6384687],
     [-0.2365917, 5.6384175],
     [-0.236582, 5.6384218],
     [-0.236558, 5.6383687],
     [-0.236568, 5.6383642],
     [-0.2365534, 5.6383318],
     [-0.2367406, 5.6382483],
     [-0.2368022, 5.6383851]]],
   [[[-0.2330788, 5.6341441],
     [-0.2328575, 5.6341827],
     [-0.2328442, 5.634112],
     [-0.2328577, 5.6341097],
     [-0.2328442, 5.6340378],
     [-0.2328319, 5.6340397],
     [-0.2328186, 5.633969],
     [-0.2330428, 5.6339314],
     [-0.2330788, 5.6341

In [79]:
get_tags_items_csv(df.iloc[5])

('znz001_buildings-item',
 'Feature',
 {'type': 'MultiPolygon',
  'coordinates': [[[[39.30435972471392, -5.72267260897133],
     [39.30437947944728, -5.722641118857668],
     [39.30440082923819, -5.722649404057447],
     [39.30442779114826, -5.722605554962872],
     [39.30445168527653, -5.722603148786133],
     [39.30448156053379, -5.722614502036288],
     [39.30449650623534, -5.722635437193972],
     [39.30485606435746, -5.722789626718194],
     [39.30485427663968, -5.722798004904199],
     [39.30499201622183, -5.722877814614503],
     [39.30496096012268, -5.722892790475567],
     [39.30491315377852, -5.722863495563576],
     [39.30484087176583, -5.722866525850526],
     [39.304480405175, -5.722689000214986],
     [39.30443697936624, -5.722753198876591],
     [39.304379613316385, -5.722720917121193],
     [39.304389153121306, -5.722686206365849],
     [39.30435972471392, -5.72267260897133]]],
   [[[39.294235267238214, -5.724875692533629],
     [39.294281828286685, -5.724806855570463],

In [80]:
# create an Item object with JSON
def create_corresponding_item(id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item):
    item_json = {
        "id": id_item,
        "type": type_item,
        "geometry": geometry_item,
        "bbox": bbox_item,
        "properties": properties_item,
        "links": links_item,
        "assets":assets_item,
    }
    it= Item(item_json)
    return it

In [81]:
os.listdir('./')

['.DS_Store',
 'data_to_STAC.csv',
 'challenge_collection_accra',
 'challenge_csv_user_friendly.ipynb',
 'tes-item.json',
 'data_to_STAC.xlsx',
 'accra',
 '.ipynb_checkpoints',
 'data_to_STAC_2.csv',
 'catalog.json',
 'monrovia',
 'pointe-noire']

In [82]:
gpd.read_file('./accra/accra_1/accra_1_buildings.geojson')

,full_id,osm_id,osm_type,building,building_l,building_m,roof_mater,amenity,roof_colou,geometry
0,r8455107,8455107,relation,yes,1,concrete;cement_block,wood;metal,None,None,"(POLYGON ((-0.2418751 5.6395062, -0.2418082 5...."
1,r8455795,8455795,relation,yes,1,metal;glass;cement_block,concrete,None,None,"(POLYGON ((-0.2368022 5.6383851, -0.2366147 5...."
2,r8456589,8456589,relation,church,3,concrete;cement_block,concrete,place_of_worship,None,"(POLYGON ((-0.2330788 5.6341441, -0.2328575 5...."
3,r8463879,8463879,relation,residential,1,metal;cement_block;concrete;wood,metal,None,None,"(POLYGON ((-0.2333627 5.6274592, -0.2331787 5...."
4,r8465922,8465922,relation,residential,1,concrete;cement_block,tile,None,None,"(POLYGON ((-0.238725 5.6373175, -0.2386065 5.6..."
5,r8486326,8486326,relation,yes,1,cement_block,metal,place_of_worship,None,"(POLYGON ((-0.2346186 5.6280171, -0.2347801 5...."
6,r8519051,8519051,relation,yes,1,glass;concrete;cement_block,metal,None,None,"(POLYGON ((-0.232821 5.6267346, -0.2327746 5.6..."
7,r8525182,8525182,relation,yes,1,cement_block,metal,None,None,"(POLYGON ((-0.2300851 5.6284862, -0.2300846 5...."
8,r8525183,8525183,relation,residential,1,cement_block,metal,None,None,"(POLYGON ((-0.2302507 5.628517, -0.2303504 5.6..."
9,r8526595,8526595,relation,yes,None,None,None,None,None,"(POLYGON ((-0.2293741 5.6275559, -0.2293035 5...."


In [83]:
id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item = get_tags_items_local('accra_1_buildings.geojson','accra_1','accra')
item = create_corresponding_item(id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item)

In [84]:
item


accra_1_buildings-item

In [85]:
item.save_as('/Users/axel/Desktop/STAC_challenge_5/tes-item.json')

accra_1_buildings-item

In [86]:
id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item = get_tags_items_csv(df.iloc[5])
item = create_corresponding_item(id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item)

### Functions for the collections

In [179]:
license_col = "CC-BY-4.0" #open, proprietary?
version_col = "1.0"
providers_col = [
    {
      "name": "WB/OCA",
      "roles": [
        "producer",
        "licensor"
      ],
      "url": "https://opencitiesproject.org"
    }
  ]
basic_keywords_col = ["challenge", "world bank"]
extent_col = {
    "spatial": [
      -180.0,
      -56.0,
      180.0,
      83.0
    ],
    "temporal": [
      "2015-06-23T00:00:00Z",
        None
    ]
  }
id_collection_basic = "challenge_collection_"
basic_description_col = "The data available for the challenge formatted in a STAC collection for the "
basic_title_col = " AoI collection"

In [180]:
def get_tags_collection(foldername):
    key = basic_keywords_col.copy()
    key.append(foldername)
    extent_col = {
    "spatial": [
      -180.0,
      -56.0,
      180.0,
      83.0
    ],
    "temporal": [
      "2015-06-23T00:00:00Z",
        None
    ]
    }
    collection_address = stac_challenge_folder +'/' +id_collection_basic+foldername+'/'+ 'catalog.json'
    links =[ {
          "rel": "self",
          "href": collection_address
        },
            {
          "rel": "root",
          "href": '../catalog.json'
        }]
    title = foldername +basic_title_col
    description = basic_description_col + foldername + ' AoI'
    id_collection = id_collection_basic + foldername
    return stac_version, id_collection, title, description, key, version_col, license_col, providers_col, extent_col, links

In [181]:
def create_corresponding_collection(stac_version, id_collection, title, description, keywords, version, license, providers, extent, links):
    # create a Collection object with JSON
    collection_json = {
    "stac_version": stac_version,
    "id": id_collection,
    "title": title ,
    "description": description,
    "collection version": version,
    "keywords":keywords,
    "license": license,
    "version": version,
    "providers":providers,
    "extent":extent,
    "links": links
    }
    col = Collection(collection_json)
    print(col.id)
    return col

### Creation of the catalog

In [270]:
stac_version = stac_version
catalog_id = 'challenge_catalog'
catalog_title = 'Challenge OpenML'
catalog_description = 'Data for the ML challenge, in the STAC format'
catalog_links = [ {
      "rel": "self",
      "href": catalog_address
    },
        {
      "rel": "root",
      "href": catalog_address
    },
]

In [271]:
# save as a root catalog
catalog_json = {
    "stac_version": stac_version,
    "id": catalog_id,
    "title": catalog_title,
    "description": catalog_description,
    "links": catalog_links   
}
catalog = Catalog(catalog_json)
catalog.save_as(catalog_address)
print('id:',catalog.id)
print('filename:',catalog.filename)
print('path:',catalog.path)
print('data:',catalog.data)

id: challenge_catalog
filename: catalog.json
path: 
data: {'stac_version': '0.6.2', 'id': 'challenge_catalog', 'title': 'Challenge OpenML', 'description': 'Data for the ML challenge, in the STAC format', 'links': [{'rel': 'self', 'href': 'catalog.json'}, {'rel': 'root', 'href': 'catalog.json'}]}


### functions to break up the creation of the collections and items:

In [272]:
def create_collection(collection):
    stac_version, id_collection, title, description, keywords, version, license, providers, extent, links = get_tags_collection(collection);
    col = create_corresponding_collection(stac_version, id_collection, title, description, keywords, version, license, providers, extent, links);
    col_address = stac_challenge_folder +'/' +"challenge_collection_"+collection+'/'+ 'catalog.json';
    col.save_as(col_address);
    id_collections.append(collection);
    return col, col_address

In [273]:
#adapt the bbox of the collection so that the new item's bbox fits in it
def adapt_bbox_col(m1_col,m2_col,M1_col,M2_col,m1,m2,M1,M2):
    if(m1<m1_col):
        m1_col = m1       
    if(m2<m2_col):
        m2_col = m2         
    if(M1>M1_col):
        M1_col = M1
    if(M2>M2_col):
        M2_col = M2
    return m1_col,m2_col,M1_col,M2_col

In [274]:
#Create and add the items that have not been previously added and are stored locally to the collection 
def add_remaining_items_from_local_to_collection(col,m1_col,m2_col,M1_col,M2_col):
    print('\n\n from folder: \n \n')
    if(collection in(os.listdir(stac_challenge_folder))):   
        #for folder in (os.listdir(stac_challenge_folder+'/'+collection)):
        for folder_1, subdirList, fileList in os.walk(stac_challenge_folder+'/'+collection):
            folder = folder_1.split('/')[-1]
            if(folder != '.DS_Store' and folder_1 != stac_challenge_folder+'/'+collection):
                create_add_local_item(id_items,collection,folder,col,m1_col,m2_col,M1_col,M2_col);

In [275]:
def create_add_local_item(id_items, collection,folder, col,m1_col,m2_col,M1_col,M2_col):
    #for file_2 in (os.listdir(stac_challenge_folder+'/'+collection+'/'+folder)):
    for dirName, subdirList, fileList in os.walk(stac_challenge_folder+'/'+collection+'/'+folder):
        for file_2 in fileList:
            if(file_2.split('.')[0] + '-item' not in id_items and file_2.split('.')[0] not in id_items and file_2 != '.DS_Store'):
                print('item from local: '+file_2.split('.')[0] + '-item.json')  
                #bbox_item,m1,m2,M1,M2 = get_bbox_row(False,[stac_challenge_folder,collection,folder,file_2])
                #m1_col,m2_col,M1_col,M2_col = adapt_bbox_col(m1_col,m2_col,M1_col,M2_col,m1,m2,M1,M2)
                id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item = get_tags_items_local(file_2,folder,collection)
                item = create_corresponding_item(id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item)
                m1_col,m2_col,M1_col,M2_col = adapt_bbox_col(m1_col,m2_col,M1_col,M2_col,bbox_item[0],bbox_item[1],bbox_item[2],bbox_item[3])
                path = folder+'/'
                filename = file_2.split('.')[0]+'-item'
                col.add_item(item, path = path, filename = filename)
                id_items.append(file_2.split('.')[0] + '-item');

In [276]:
#create the collections that were not listed in csv, then create their items, and add them to the collection
def create_and_add_collections_from_local(id_collections, id_items):
    print('\n \n add collections stored locally but not listed in the csv \n \n');
    for collection in(os.listdir(stac_challenge_folder)):
        if('challenge_collection' not in collection and 'data_to_STAC' not in collection and collection != '.DS_Store' and collection !='challenge_csv_user_friendly.ipynb' and collection !='.ipynb_checkpoints' and '.json' not in collection and 'challenge_collection' not in collection):
            if(collection not in id_collections):
                print('collection from folder:',collection);
                col, col_address = create_collection(collection)
                catalog.add_catalog(col)
                #for folder in (os.listdir(stac_challenge_folder+'/'+collection)):
                for folder_1, subdirList, fileList in os.walk(stac_challenge_folder+'/'+collection):
                    folder = folder_1.split('/')[-1];
                    if(folder != '.DS_Store' and folder_1!=stac_challenge_folder+'/'+collection):
                        print(folder)
                        file = fileList[0]
                        if('.DS_Store' in file):
                            file = fileList[1]
                        bbox = get_bbox(stac_challenge_folder+'/'+collection+'/'+folder+'/'+file)
                        m1,m2,M1,M2 = bbox_item[0],bbox_item[1],bbox_item[2],bbox_item[3]
                        m1_col = bbox[0]
                        m2_col = bbox[1]
                        M1_col = bbox[2]
                        M2_col = bbox[3]
                        create_add_local_item(id_items,collection,folder, col,m1_col,m2_col,M1_col,M2_col); 
                bbox_col = [m1_col,m2_col,M1_col,M2_col]
                col.data['extent']['spatial'] = bbox_col
                col.save_as(col_address) 
                cols[collection] = col;

In [277]:
#Create and add the items that are listed in the csv to the collection
def create_and_add_items_to_collection_from_csv(df_col,m1_col,m2_col,M1_col,M2_col, col, collection):
    print('\n\n from csv: \n \n')
    for i,row in df_col.iterrows():
        file_i = str(row['id']) + '.geojson'
        print(file_i)
        print('good0')
        bbox_item = get_bbox_csv(row);
        print('good1')
        m1,m2,M1,M2 = bbox_item[0],bbox_item[1],bbox_item[2],bbox_item[3]
        m1_col,m2_col,M1_col,M2_col = adapt_bbox_col(m1_col,m2_col,M1_col,M2_col,m1,m2,M1,M2)
        #item = create_item(True,row, collection)
        id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item = get_tags_items_csv(row)
        item = create_corresponding_item(id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item)
        path = row['part']+'/'
        filename = row['id']
        col.add_item(item, path = path, filename = filename)
        id_items.append(row['id']);

In [278]:
def get_tags_collection(foldername):
    key = basic_keywords_col.copy()
    key.append(foldername)
    extent_col = {
    "spatial": [
      -180.0,
      -56.0,
      180.0,
      83.0
    ],
    "temporal": [
      "2015-06-23T00:00:00Z",
        None
    ]
    }
    collection_address = stac_challenge_folder +'/' +id_collection_basic+foldername+'/'+ 'catalog.json'
    links =[ {
          "rel": "self",
          "href": collection_address
        },
            {
          "rel": "root",
          "href": '../catalog.json'
        }]
    title = foldername +basic_title_col
    description = basic_description_col + foldername + ' AoI'
    id_collection = id_collection_basic + foldername
    return stac_version, id_collection, title, description, key, version_col, license_col, providers_col, extent_col, links

In [ ]:
id_collections = []
id_items = []
cols = {};

for collection in df['collection'].unique():
    print('collection:',collection);
    col, col_address = create_collection(collection);
    catalog.add_catalog(col)
    df_col = df[df['collection'] == collection];
    bbox = get_bbox_csv(df_col.iloc[0]);
    m1,m2,M1,M2 = bbox[0],bbox[1],bbox[2],bbox[3];
    m1_col = bbox[0]
    m2_col = bbox[1]
    M1_col = bbox[2]
    M2_col = bbox[3]
    
    create_and_add_items_to_collection_from_csv(df_col,m1_col,m2_col,M1_col,M2_col,col, collection)
    add_remaining_items_from_local_to_collection(col,m1_col,m2_col,M1_col,M2_col)
    
    bbox_col = [m1_col,m2_col,M1_col,M2_col]
    col.data['extent']['spatial'] = bbox_col
    col.save_as(col_address) 
    cols[collection] = col;
create_and_add_collections_from_local(id_collections, id_items)
catalog.save_as(catalog_address)

collection: accra
challenge_collection_accra


 from csv: 
 

accra_1_buildings.geojson
good0
good1
accra_1_drains.geojson
good0
good1
accra_1_roads.geojson
good0
good1


 from folder: 
 

item from local: accra_2_roads-item.json
item from local: accra_2_drains-item.json
item from local: accra_2_buildings-item.json


In [176]:
gdf = gpd.read_file(df.iloc[5]['asset:vector:href'])

In [177]:
gdf

,pk_0,pk,pkuid,building,unfinished,problemati,changeset,area,condition,geometry
0,1,None,None,None,None,None,None,None,Complete,"POLYGON ((39.30435972471392 -5.72267260897133,..."
1,2,None,None,None,None,None,None,None,Complete,POLYGON ((39.29423526723821 -5.724875692533629...
2,3,None,None,None,None,None,None,None,Complete,"POLYGON ((39.29415344989369 -5.72492420284619,..."
3,4,None,None,None,None,None,None,None,Complete,POLYGON ((39.29405249930165 -5.724939812360279...
4,5,None,None,None,None,None,None,None,Complete,POLYGON ((39.29396645282554 -5.724895576762622...
5,6,None,None,None,None,None,None,None,Complete,POLYGON ((39.30476742896555 -5.722933732356477...
6,7,None,None,None,None,None,None,None,Incomplete,POLYGON ((39.29558174481707 -5.729395965960126...
7,8,None,None,None,None,None,None,None,Complete,POLYGON ((39.29569166963447 -5.729404884858749...
8,9,None,None,None,None,None,None,None,Complete,POLYGON ((39.29543730009508 -5.729637783698991...
9,10,None,None,None,None,None,None,None,Complete,"POLYGON ((39.294116120645 -5.724354569435363, ..."
